# Project task 04:  Restaurant ranking

In [1]:
import numpy as np
import scipy.sparse as sp
import time

The goal of this task is to rank restaurants using the **PageRank** algorithm. You are given a directed weighted graph where each node represents one restaurant. The edges in this graph are based on users reviews.

Additionally for each restaurant you are given the categories it belongs to, i.e. 'Mexican', 'Italian', etc. Note that each restaurant can belong to multiple categories.

Considering these categories as topics you will perform **topic-sensitive PageRank**, enabling you to e.g. find the top 10 'Mexican' restaurants.

## 1. Load data

* The graph is stored as a sparse adjacency matrix $A$
* The categories are stored in a binary sparse matrix $C$, with $C_{ij}=1$ indicating that restaurant $i$ belongs to category $j$
* We also provide you with a dictionary mapping each category to its corresponding column index in $C$
* The name of each restaurant is provided as a list, with the i-th element in the list corresponding to the i-th node in the graph

In [2]:
A = sp.load_npz('restaurant_graph.npz')
A

<7073x7073 sparse matrix of type '<class 'numpy.float64'>'
	with 1682844 stored elements in Compressed Sparse Row format>

In [3]:
C = sp.load_npz('restaurant_categories.npz')
C

<7073x138 sparse matrix of type '<class 'numpy.float64'>'
	with 19047 stored elements in Compressed Sparse Row format>

In [4]:
categories = np.load('categories.npy').tolist()
categories['Mexican'], categories['Chinese']

(3, 14)

In [5]:
names = np.load('restaurant_names.npy')
names[:3]

array(['Alize Catering', 'Chula Taberna Mexicana', 'Sunnyside Grill'],
      dtype='<U50')

In [6]:
assert A.shape[0] == len(names) == C.shape[0]
assert C.shape[1] == len(categories)

In [7]:
B = C.getcol(categories['Mexican'])
idx2,_ = B.nonzero()
indeces = C.nonzero()
idx1 = np.split(C.indices, C.indptr[1:-1])
print(idx2)

[   1   11   34   39   47  135  144  172  181  184  206  276  289  387
  388  424  432  446  467  553  564  579  583  599  625  643  669  727
  789  817  905  930 1033 1046 1131 1166 1273 1275 1286 1293 1340 1352
 1382 1402 1416 1486 1540 1561 1569 1614 1733 1743 1756 1794 1886 1898
 1902 1909 1924 1925 1926 1941 2031 2043 2069 2070 2101 2146 2206 2226
 2311 2329 2390 2496 2537 2538 2572 2573 2576 2579 2591 2608 2628 2734
 2770 2815 2839 2847 2852 2880 2887 2897 2949 2955 3018 3149 3165 3211
 3221 3235 3239 3261 3274 3311 3332 3341 3364 3416 3430 3439 3441 3447
 3456 3459 3460 3464 3479 3492 3545 3584 3606 3617 3693 3728 3791 3794
 3849 3874 3885 3919 3932 3935 3939 3942 3975 3988 3992 4007 4008 4019
 4020 4045 4049 4170 4191 4254 4268 4320 4321 4332 4343 4382 4395 4402
 4417 4418 4426 4478 4542 4569 4571 4587 4598 4604 4609 4615 4624 4689
 4695 4796 4800 4815 4886 4925 4946 4958 4999 5019 5020 5052 5116 5157
 5187 5230 5233 5271 5300 5315 5324 5391 5442 5480 5498 5534 5546 5574
 5621 

In [8]:
## scaling test to extract non-zero nodes
prob = 1- C.nnz/(C.shape[0]*C.shape[1])
shape = (10000, 10000)
M = sp.csr_matrix(np.random.choice(2, shape, p=[prob, 1-prob]))
print(M.shape)
#print(C)
topics = ['Mexican', 'Chinese', 'Korean'] 

start = time.time()

col_idx = [categories[topic] for topic in topics]
C1 = M[:, col_idx]
idx,_ = C1.nonzero()
print(idx)
print('Time: ', time.time()-start)

(10000, 10000)
[   4   32   43   78   92  113  157  216  235  243  258  281  286  325
  341  355  367  397  398  400  405  424  424  440  450  452  469  474
  475  487  504  508  512  522  544  545  563  567  575  587  593  611
  617  679  694  732  734  741  763  799  829  843  877  892  900  902
  903  929  946  952  960  981 1032 1039 1090 1097 1111 1122 1126 1133
 1146 1162 1177 1189 1225 1226 1233 1235 1248 1259 1273 1293 1313 1315
 1320 1347 1373 1402 1415 1435 1457 1472 1507 1553 1565 1570 1580 1588
 1597 1609 1620 1620 1631 1666 1678 1707 1737 1738 1742 1747 1757 1791
 1850 1851 1855 1862 1893 1922 1944 1949 1955 2036 2047 2067 2101 2121
 2168 2182 2183 2215 2227 2274 2287 2306 2307 2327 2333 2333 2380 2384
 2394 2395 2411 2415 2422 2430 2443 2447 2484 2510 2543 2583 2595 2623
 2632 2638 2643 2646 2692 2715 2743 2747 2753 2761 2784 2784 2787 2811
 2835 2891 2898 2945 2946 2946 2998 3013 3029 3038 3051 3052 3056 3073
 3092 3128 3148 3154 3201 3243 3249 3256 3274 3280 3297 3299 3

 ## 2. Determine the teleport set
 

Given a list of topics of intereset, i.e. `['Mexican', 'Italian', ...]`, implement a helper function to return all the restaurants that belong to **at least one** of these topics. These restaurants will become part of the teleport set in topic-sensitive PageRank.

In [9]:
def teleport_set(C, topics, categories):
    """
    Finds the teleport set consisting of restaurants that belong to at least one of the specified topics.
    
    Parameters
    ----------
    C             : sp.spmatrix, shape [num_restaurants, num_categories]
                    Binary matrix encoding which restaurants belongs to which categories.
    topics        : List[string]
                    List of topics of interest.
    categories    : dict(string, int)
                    Dictionary mapping each category to its corresponding column index in C.
        
    Returns
    -------
    teleport_idx : np.array, shape [S]
                   The indicies of the nodes in the teleport set.
    """
    #### YOUR CODE ####
    
    col_idx = [categories[topic] for topic in topics]
    C1 = C[:, col_idx]
    #print(len(C1.nonzero()[0]))

    #teleport_idx = np.array(C1.nonzero()[0]).ravel()
    teleport_idx,_ = C1.nonzero()    
    print(teleport_idx.shape)
    return teleport_idx

 ## 3. Implement topic-sensitive PageRank

In [86]:
def page_rank(A, beta, teleport_idx=None, eps=1e-12):
    """
    Implements topic-sensitive PageRank using power iteration and sparse matrix operations.
    
    Parameters
    ----------
    A           : sp.spmatrix, shape [num_restaurants, num_restaurants]
                  The adjacency matrix representing the graph of restaurants.
    beta        : float, 
                  0 < beta < 1, (1-beta) is the probabilty of teleporting to the nodes in the teleport set
    teleport_idx: np.array, shape [S]
                  The indicies of the nodes in the teleport set. If it equals to None
                  it means runs standard PageRank, i.e. all nodes are in the teleport set.
    
    Returns
    -------
    r          : np.array, shape [num_restaurants]
                 The page rank vector containing the page rank scores for each restaurant.
    """
    
    """
    #### YOUR CODE ####
    num_restaurants, _ = A.shape
    
    #Normalize columns to make it stochastic
    A /= A.sum(axis=0)
    A = sp.csr_matrix(A)
    
    #Initialize teleport vector
    if teleport_idx is None:
        teleport_vec = np.ones(num_restaurants) / num_restaurants
    else:
        num_S, = teleport_idx.shape
        teleport_vec = np.zeros(num_restaurants)
        teleport_vec[teleport_idx] = 1.0 / num_S
    
    #Initialize r
    r = np.ones(num_restaurants) / num_restaurants
    r_old = np.ones(num_restaurants)
    
    #Power iteration
    while np.linalg.norm(r_old - r) > eps:
        r_old = r
        r = beta * A.dot(r) + (1-beta) * teleport_vec
    
    r = np.array(r)
    return r
    """
    
    #### YOUR CODE ####
    start = time.time()
    num_r,_ = A.shape
    
    A /= A.sum(axis=0)
    A = sp.csr_matrix(A)

    r1 = np.ones(num_r)/num_r
    r = np.zeros(num_r)
    
    if teleport_idx is None:
        teleport_vec = np.ones(num_r)/num_r
        while np.linalg.norm(r1-r)>eps:
            r = beta*A.dot(r1) + (1-beta)*teleport_vec
            r1 = r
    else:
        num_tel = teleport_idx.shape[0]
        teleport_vec = np.zeros(num_r)
        teleport_vec[teleport_idx] = 1.0/num_tel
        teleport_vec = sp.csr_matrix(teleport_vec)
        r1 = sp.csr_matrix(r1).T
        r = sp.csr_matrix(r).T
        print(r.shape, r1.shape, A.shape)
        print(sp.linalg.norm(A))
        while sp.linalg.norm(r1-r)>eps:
            r = beta*A.dot(r1) + (1-beta)*teleport_vec
            r1 = r

    r = np.array(r)
    print('Time:', time.time()-start)
    return r

### 3.1 Calculate the standard PageRank scores and print the names of the top 5 restaurants overall

In [87]:
idx_to_category = {v:k for k, v in categories.items()}

In [88]:
r = page_rank(A=A, beta=0.6, teleport_idx=None)

for i, x in enumerate(r.argsort()[-5:]):
    print(i+1, names[x], '\n  Categories: ', [idx_to_category[cat] for cat in C[x].nonzero()[1]])

Time: 0.7966930866241455
1 Edulis 
  Categories:  ['Sandwiches']
2 Congee Me 
  Categories:  ['Korean']
3 Sushi Making For the Soul 
  Categories:  ['Japanese']
4 Thai Indeed 
  Categories:  ['Chinese']
5 Happy Tummy Filipino Cuisine 
  Categories:  ['Chinese']


### 3.2 Calculate the topic-sensitive PageRank scores and print the names of top 5 Mexican restaurants

In [89]:
teleport_idx = teleport_set(C, ['Mexican'], categories)
r = page_rank(A=A, beta=0.6, teleport_idx=teleport_idx)

for i, x in enumerate(r.argsort()[-5:]):
    print(i+1, names[x], '\n  Categories: ', [idx_to_category[cat] for cat in C[x].nonzero()[1]])

(242,)
(7073, 1) (7073, 1) (7073, 7073)


AttributeError: module 'scipy.sparse' has no attribute 'linalg'

### 3.3 Calculate the topic-sensitive PageRank scores and print the names of top 5 Italian or French restaurants


In [137]:
teleport_idx = teleport_set(C, ['Italian', 'French'], categories)
r = page_rank(A=A, beta=0.6, teleport_idx=teleport_idx)

for i, x in enumerate(r.argsort()[-5:]):
    print(i+1, names[x], '\n  Categories: ', [idx_to_category[cat] for cat in C[x].nonzero()[1]])

(731,)


NameError: name 'page_rank' is not defined